# CHICAGO CENSUS DATA  ANALYST
## Using SQL & Python (Jupyter Notebook)

### Establish A Sequel Connection

In [1]:
import ibm_db
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "wyt93087"        # e.g. "abc12345"
dsn_pwd = "9YI2h2kUOoqoIstu"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "32733"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  wyt93087 on host:  54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [2]:
import ibm_db_dbi
import pandas as pd
sql = "SELECT NAME FROM sysibm.systables WHERE creator='WYT93087'"
pconn = ibm_db_dbi.Connection(conn)
df = pd.read_sql(sql, pconn)
print(df)

                   NAME
0                TRUCKS
1  CHICAGOPUBLICSCHOOLS
2      CHICAGOCRIMEDATA
3     CHICAGOCENSUSDATA


### The total number of crimes recorded in the CRIME table.

In [3]:
sql = "SELECT COUNT(*) FROM CHICAGOCRIMEDATA"
df = pd.read_sql(sql, pconn)

"The total number of crimes recorded in the CRIME table is "+ str(int(df.iloc[0]))+"."

'The total number of crimes recorded in the CRIME table is 533.'

### The List community areas with per capita income less than 11000.

In [4]:
sql = "SELECT DISTINCT(COMMUNITY_AREA_NAME),PER_CAPITA_INCOME FROM CHICAGOCENSUSDATA WHERE PER_CAPITA_INCOME < 11000 "
df = pd.read_sql(sql, pconn)
print(df)
dflist = df['COMMUNITY_AREA_NAME'].tolist()
"List community areas with per capita income less than 11000 are "+', '.join(dflist)+'.'


  COMMUNITY_AREA_NAME  PER_CAPITA_INCOME
0         Fuller Park              10432
1           Riverdale               8201
2      South Lawndale              10402
3  West Garfield Park              10934


'List community areas with per capita income less than 11000 are Fuller Park, Riverdale, South Lawndale, West Garfield Park.'

### The List all case numbers for crimes involving minors.

In [5]:
sql = "SELECT ID,PRIMARY_TYPE,DESCRIPTION FROM CHICAGOCRIMEDATA WHERE DESCRIPTION LIKE '%CHILD%' OR PRIMARY_TYPE LIKE '%CHILD%' "
df = pd.read_sql(sql, pconn)
print(df)
"All case numbers for crimes involving minors are: "+"ID-"+str(df['ID'][0])+", "+"ID-"+str(df['ID'][1])+" and "+"ID-"+str(df['ID'][2])+"."

        ID     PRIMARY_TYPE                     DESCRIPTION
0  5766654  OFFENSE INVOLVI  AGG SEX ASSLT OF CHILD FAM MBR
1  6986273  OFFENSE INVOLVI   SEX ASSLT OF CHILD BY FAM MBR
2  5276766       KIDNAPPING        CHILD ABDUCTION/STRANGER


'All case numbers for crimes involving minors are: ID-5766654, ID-6986273 and ID-5276766.'

### The List all kidnapping crimes involving a child?

In [6]:
sql = "SELECT * FROM CHICAGOCRIMEDATA WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'"
df = pd.read_sql(sql, pconn)
print(df)
"There is 1 kidnapping crimes involving a child as listed on top."

        ID CASE_NUMBER        DATE                 BLOCK  IUCR PRIMARY_TYPE  \
0  5276766    HN144152  2007-01-26  050XX W VAN BUREN ST  1792   KIDNAPPING   

                DESCRIPTION LOCATION_DESCRIPTION ARREST DOMESTIC  ...  \
0  CHILD ABDUCTION/STRANGER               STREET  FALSE    FALSE  ...   

   DISTRICT  WARD  COMMUNITY_AREA_NUMBER  FBICODE X_COORDINATE  Y_COORDINATE  \
0        15    29                     25       20      1143050       1897546   

   YEAR   LATITUDE  LONGITUDE                       LOCATION  
0  2007  41.874908 -87.750249  (41.874908413, -87.750249307)  

[1 rows x 21 columns]


'There is 1 kidnapping crimes involving a child as listed on top.'

### What kind of crimes were recorded at schools?

In [7]:
sql = "SELECT DISTINCT(PRIMARY_TYPE),LOCATION_DESCRIPTION FROM CHICAGOCRIMEDATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'"
df = pd.read_sql(sql, pconn)
print(df)
dflist = df['PRIMARY_TYPE'].tolist()
lowerlist = []
print(dflist)
for i in dflist:
    lowerlist += [i.lower()]
"The kind of crimes were recorded at schools are "+', '.join(lowerlist)+'.'

      PRIMARY_TYPE       LOCATION_DESCRIPTION
0  PUBLIC PEACE VI  SCHOOL, PRIVATE, BUILDING
1          BATTERY   SCHOOL, PUBLIC, BUILDING
2        NARCOTICS   SCHOOL, PUBLIC, BUILDING
3  PUBLIC PEACE VI   SCHOOL, PUBLIC, BUILDING
4          ASSAULT    SCHOOL, PUBLIC, GROUNDS
5          BATTERY    SCHOOL, PUBLIC, GROUNDS
6  CRIMINAL DAMAGE    SCHOOL, PUBLIC, GROUNDS
7  CRIMINAL TRESPA    SCHOOL, PUBLIC, GROUNDS
8        NARCOTICS    SCHOOL, PUBLIC, GROUNDS
['PUBLIC PEACE VI', 'BATTERY', 'NARCOTICS', 'PUBLIC PEACE VI', 'ASSAULT', 'BATTERY', 'CRIMINAL DAMAGE', 'CRIMINAL TRESPA', 'NARCOTICS']


'The kind of crimes were recorded at schools are public peace vi, battery, narcotics, public peace vi, assault, battery, criminal damage, criminal trespa, narcotics.'

### The List the average safety score for all types of schools.

In [8]:
sql = "SELECT AVG(SAFETY_SCORE) FROM CHICAGOPUBLICSCHOOLS"
df = pd.read_sql(sql, pconn)
print(df)
'The average safety score for schools in Chicago is '+ str(int(df.iloc[0]))+"."

    1
0  49


'The average safety score for schools in Chicago is 49.'

### The List of 5 community areas with highest % of households below poverty line

In [9]:
sql = "SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CHICAGOCENSUSDATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5"
df = pd.read_sql(sql, pconn)
print(df)
dflist = df['COMMUNITY_AREA_NAME'].tolist()
'The 5 community areas with highest% of households below poverty line are: '+', '.join(dflist)+'.'

  COMMUNITY_AREA_NAME  PERCENT_HOUSEHOLDS_BELOW_POVERTY
0           Riverdale                              56.5
1         Fuller Park                              51.2
2           Englewood                              46.6
3      North Lawndale                              43.1
4  East Garfield Park                              42.4


'The 5 community areas with highest% of households below poverty line are: Riverdale, Fuller Park, Englewood, North Lawndale, East Garfield Park.'

### Which community area is most crime prone?

In [10]:
#Method 1 Using Pandas Index & 2 SQL query:
sql = "SELECT COMMUNITY_AREA_NUMBER,COUNT(COMMUNITY_AREA_NUMBER) AS NUM_CASES FROM CHICAGOCRIMEDATA GROUP BY(COMMUNITY_AREA_NUMBER) ORDER BY NUM_CASES DESC LIMIT 5"
df = pd.read_sql(sql, pconn)
print(df)
crime_area_num =int(df['COMMUNITY_AREA_NUMBER'].iloc[0])
sql = "SELECT COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER FROM CHICAGOCENSUSDATA WHERE COMMUNITY_AREA_NUMBER = '%s'" % crime_area_num
df = pd.read_sql(sql, pconn)
print(df)
crime_area_num =str(int(df['COMMUNITY_AREA_NUMBER'].iloc[0]))
crime_area_name =str(df['COMMUNITY_AREA_NAME'].iloc[0])
'The community area is most crime prone is '+crime_area_name+' which area number is '+crime_area_num+'.'

   COMMUNITY_AREA_NUMBER  NUM_CASES
0                     25         43
1                     23         22
2                     68         21
3                     28         16
4                     29         16
  COMMUNITY_AREA_NAME  COMMUNITY_AREA_NUMBER
0              Austin                     25


'The community area is most crime prone is Austin which area number is 25.'

In [11]:
#Method 2 Using Join SQL query in single query:
sql = "SELECT c.COMMUNITY_AREA_NUMBER,d.COMMUNITY_AREA_NAME,COUNT(c.COMMUNITY_AREA_NUMBER) AS NUM_CASES  FROM CHICAGOCRIMEDATA AS c JOIN CHICAGOCENSUSDATA AS d ON c.COMMUNITY_AREA_NUMBER = d.COMMUNITY_AREA_NUMBER GROUP BY c.COMMUNITY_AREA_NUMBER,d.COMMUNITY_AREA_NAME ORDER BY NUM_CASES DESC LIMIT 5"
#I got to say method 1 is alot easier.
df = pd.read_sql(sql, pconn)
print(df)
crime_area_num =str(int(df['COMMUNITY_AREA_NUMBER'].iloc[0]))
crime_area_name =str(df['COMMUNITY_AREA_NAME'].iloc[0])
'The community area is most crime prone is '+crime_area_name+' which area number is '+crime_area_num+'.'

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  NUM_CASES
0                     25              Austin         43
1                     23       Humboldt park         22
2                     68           Englewood         21
3                     28      Near West Side         16
4                     29      North Lawndale         16


'The community area is most crime prone is Austin which area number is 25.'

### Using Sub-query to find the name of the community area with highest hardship index

In [12]:
sql = "SELECT COMMUNITY_AREA_NAME,HARDSHIP_INDEX FROM (SELECT * FROM CHICAGOCENSUSDATA WHERE HARDSHIP_INDEX IS NOT NULL) ORDER BY HARDSHIP_INDEX DESC LIMIT 5"
#sub-query is very unnecessary to solve this problem..
df = pd.read_sql(sql, pconn)
print(df)
name = df['COMMUNITY_AREA_NAME'].iloc[0]
'The name of the community area with highest hardship index is '+name+'.'

  COMMUNITY_AREA_NAME  HARDSHIP_INDEX
0           Riverdale              98
1         Fuller Park              97
2      South Lawndale              96
3           Englewood              94
4           Gage Park              93


'The name of the community area with highest hardship index is Riverdale.'

### Using a sub-query to determine the Community Area Name with most number of crimes?

In [13]:
sql = "SELECT COMMUNITY_AREA_NAME FROM CHICAGOCENSUSDATA WHERE COMMUNITY_AREA_NUMBER IN (SELECT COMMUNITY_AREA_NUMBER from CHICAGOCRIMEDATA  group by COMMUNITY_AREA_NUMBER order by COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1)"
df = pd.read_sql(sql, pconn)
print(df)
crime_area_name =str(df['COMMUNITY_AREA_NAME'].iloc[0])
'The community area is most crime prone is '+crime_area_name+'.'

  COMMUNITY_AREA_NAME
0              Austin


'The community area is most crime prone is Austin.'

### End Sequel Connection

In [14]:
ibm_db.close(conn)

True